# Advanced SQL

## Lecture Objectives

#### Part 1 SQL with python (psycopg2):

- Connect to a database from within a python program and run queries
- Understand psycopg2's cursors and commits
- Generate dynamic queries

#### Part 2 RDBMS the why and how:

 - Why an RDBMS?
 - The database lifecycle (and where the data scientist fits!)
 - Primary keys, foriegn keys, and the ERD
 - Creating and maintaining tables
 - Order of Evaluation of a SQL SELECT Statement
 - Transactions
 - SQL Execution Plan

## Part 1 Combining SQL and Python

Often you will find yourself working with data that are only accessable through SQL.  However, your machine-learning capabilities are built in Python.  To resolve this issue, we can simply set up a connection from Python to the SQL database to bring the data to us.

## Why do we care?

- SQL-based databases are extremely common in almost all industry environments
- Can leverage the benefit of SQL's structure and scalability, while maintaining the flexibility of Python
- Very useful for scaled data pipelines, pre-cleaning, data exploration
- Allows for dynamic query generation and hence automations

## psycopg2

- A Python library that allows for connections with PostgresSQL databases to easily query and retrieve data for analysis.
- [Documentation--Includes Installation Instructions](http://initd.org/psycopg/docs/install.html)
- In addition to what's listed in the documentation, if you have the anaconda distribution of Python 
- There are similar packages for other flavors of SQL that work much the same way

To install using anaconda try \
`conda install psycopg2`\
If that does not work for your environment try:\
`pip install psycopg2-binary`



## General Workflow

1. Establish connection to Postgres database using psycopg2
2. Create a cursor
3. Use the cursor to execute SQL queries and retrieve data
4. Commit SQL actions
4. Close the cursor and connection

# Walkthrough 1: Creating a database from Python

In [133]:
#before starting with this notebook, make sure you set your docker path. If running this locally, just use your local path.
docker_path = '/home/data/galv/lec2/lectures/sql-advanced'

### Connect to the database
- Connections must be established using an existing database, username, database IP/URL, and maybe passwords
- If you need to create a database, you can first connect to Postgres using the dbname 'postgres' to initialize

In [134]:
import psycopg2
conn = psycopg2.connect(dbname='postgres', host='localhost', port = 8997, user="postgres", password="galvanize")

In [135]:
type(conn)

psycopg2.extensions.connection

### Instantiate the Cursor

- A cursor is a control structure that enables traversal over the records in a database
- Executes and fetches data
- When the cursor points at the resulting output of a query, it can only read each observation once.  If you choose to see a previously read observation, you must rerun the query. 
- Can be closed without closing the connection

In [136]:
cur = conn.cursor()

### Commits

- Data changes are not actually stored until you choose to commit
- You can choose to have automatic commit by using ` autocommit = True`
- When connecting directly to the Postgres Server to initiate server level commands such as creating a database, you must use the `autocommit = True` option since Postgres does not have "temporary" transactions at the database level

In [137]:
conn.autocommit = True

###  Create a database

In [138]:
cur.execute('DROP DATABASE IF EXISTS temp;')
cur.execute('CREATE DATABASE temp;')

### Disconnect from the cursor and database
- Cursors and Connections must be closed using .close() or else Postgres will lock certain operation on the database/tables to connection is severed. 

In [139]:
cur.close() # This is optional
conn.close() # Closing the connection also closes all cursors

# Walkthrough 2: Using the new database

### Connect to our database

In [140]:
conn = psycopg2.connect(dbname='temp', host='localhost', port = 8997, user="postgres", password="galvanize")

In [141]:
cur = conn.cursor()

### Create a new table

In [142]:
query = '''
        CREATE TABLE logins (
            userid integer, 
            tmstmp timestamp, 
            type varchar(10)
        );
        '''
cur.execute(query)

### Insert data into new table

`os` is needed to get the current directory and, later, dynamically identify the files we need to insert using listdir.

In [143]:
import os

In [144]:
local_path = os.getcwd()

In [145]:
!pwd

/Users/alexblitstein/galv/lec2/lectures/sql-advanced


In [146]:
query = '''
        COPY logins 
        FROM '{0}/logins_data/logins01.csv' 
        DELIMITER ',' 
        CSV;
        '''.format(docker_path)
print(query)
cur.execute(query)


        COPY logins 
        FROM '/home/data/galv/lec2/lectures/sql-advanced/logins_data/logins01.csv' 
        DELIMITER ',' 
        CSV;
        


### Run a query to get 30 records from our data

In [147]:
query = '''
        SELECT *
        FROM logins
        LIMIT 30;
        '''
cur.execute(query)

### Lets look at our data one line at a time

In [148]:
cur.fetchone()

(579, datetime.datetime(2013, 11, 20, 3, 20, 6), 'mobile')

### Many lines at a time

In [149]:
#fetchmany(n) to get n rows
cur.fetchmany(10)

[(823, datetime.datetime(2013, 11, 20, 3, 20, 49), 'web'),
 (953, datetime.datetime(2013, 11, 20, 3, 28, 49), 'web'),
 (612, datetime.datetime(2013, 11, 20, 3, 36, 55), 'web'),
 (269, datetime.datetime(2013, 11, 20, 3, 43, 13), 'web'),
 (799, datetime.datetime(2013, 11, 20, 3, 56, 55), 'web'),
 (890, datetime.datetime(2013, 11, 20, 4, 2, 33), 'mobile'),
 (330, datetime.datetime(2013, 11, 20, 4, 54, 59), 'mobile'),
 (628, datetime.datetime(2013, 11, 20, 4, 57, 22), 'mobile'),
 (398, datetime.datetime(2013, 11, 20, 5, 3, 19), 'mobile'),
 (482, datetime.datetime(2013, 11, 20, 5, 4, 43), 'mobile')]

### Or everything at once

In [150]:
#fetchall() grabs all remaining rows
results = cur.fetchall()

In [151]:
print(len(results))
print(results)

19
[(581, datetime.datetime(2013, 11, 20, 5, 12, 3), 'mobile'), (370, datetime.datetime(2013, 11, 20, 5, 26, 46), 'mobile'), (230, datetime.datetime(2013, 11, 20, 5, 28, 29), 'web'), (596, datetime.datetime(2013, 11, 20, 5, 28, 36), 'web'), (274, datetime.datetime(2013, 11, 20, 5, 43, 8), 'mobile'), (581, datetime.datetime(2013, 11, 20, 5, 47, 10), 'web'), (417, datetime.datetime(2013, 11, 20, 5, 54, 37), 'mobile'), (185, datetime.datetime(2013, 11, 20, 5, 56, 22), 'mobile'), (371, datetime.datetime(2013, 11, 20, 5, 58, 35), 'mobile'), (133, datetime.datetime(2013, 11, 20, 5, 59, 7), 'web'), (621, datetime.datetime(2013, 11, 20, 6, 1, 46), 'web'), (306, datetime.datetime(2013, 11, 20, 6, 3, 23), 'mobile'), (509, datetime.datetime(2013, 11, 20, 6, 4, 43), 'web'), (505, datetime.datetime(2013, 11, 20, 6, 9, 52), 'web'), (678, datetime.datetime(2013, 11, 20, 6, 34, 18), 'web'), (889, datetime.datetime(2013, 11, 20, 6, 36, 32), 'mobile'), (202, datetime.datetime(2013, 11, 20, 6, 43, 33), '

In [152]:
#cur.scroll(1)

### You can even iterate over the cursor

In [153]:
cur.execute(query)
for record in cur:
    print("{}: user {} logged in via {}".format(record[1], record[0], record[2]))

2013-11-20 03:20:06: user 579 logged in via mobile
2013-11-20 03:20:49: user 823 logged in via web
2013-11-20 03:28:49: user 953 logged in via web
2013-11-20 03:36:55: user 612 logged in via web
2013-11-20 03:43:13: user 269 logged in via web
2013-11-20 03:56:55: user 799 logged in via web
2013-11-20 04:02:33: user 890 logged in via mobile
2013-11-20 04:54:59: user 330 logged in via mobile
2013-11-20 04:57:22: user 628 logged in via mobile
2013-11-20 05:03:19: user 398 logged in via mobile
2013-11-20 05:04:43: user 482 logged in via mobile
2013-11-20 05:12:03: user 581 logged in via mobile
2013-11-20 05:26:46: user 370 logged in via mobile
2013-11-20 05:28:29: user 230 logged in via web
2013-11-20 05:28:36: user 596 logged in via web
2013-11-20 05:43:08: user 274 logged in via mobile
2013-11-20 05:47:10: user 581 logged in via web
2013-11-20 05:54:37: user 417 logged in via mobile
2013-11-20 05:56:22: user 185 logged in via mobile
2013-11-20 05:58:35: user 371 logged in via mobile
2013

# Dynamic Queries

- A Dynamic Query is a query that generates based on context.


### Example

We have 8 login csv files that we need to insert into the logins table.  Instead of doing a COPY FROM query 8 times, we should utilize Python (or any future languages) to make this more efficient.  This is possible due to tokenized strings.

### First lets get an idea of how many records we start with

In [154]:
cur.execute('SELECT count(*) FROM logins;')
record_count = cur.fetchone()[0]

In [155]:
record_count

10000

In [156]:
type(record_count)

int

In [157]:
conn.commit()

### Create a query template and determine file path for imports


## **[WARNING: BEWARE OF SQL INJECTION](http://initd.org/psycopg/docs/usage.html)**

### What is an SQL Injection Attack?

from W3schools.com:

 - SQL injection is a code injection technique that might destroy your database.
 - SQL injection is **one of the most common** web hacking techniques.
 - SQL injection is the placement of malicious code in SQL statements, via web page input.
 
SQL injection usually occurs when you ask a user for input, like their username/userid, and instead of a name/id, the user gives you an SQL statement that you will **unknowingly run on your database.**

Here is an example:

In [158]:
date_cut = "2014-08-01; DROP TABLE logins" # The user enters a date in a field on a web form

horribly_risky = "SELECT * FROM logins WHERE tmstmp > %s;" % date_cut

print(horribly_risky)

SELECT * FROM logins WHERE tmstmp > 2014-08-01; DROP TABLE logins;


To avoid SQL Injections **NEVER** use + or % or .format to reformat strings to be used with .execute

In [159]:
num = 579
terribly_unsafe = "SELECT * FROM logins WHERE userid = " + str(num) + ";"
print(terribly_unsafe)


date_cut = "2014-08-01"
horribly_risky = "SELECT * FROM logins WHERE tmstmp > %s;" % date_cut
print(horribly_risky)
## Python is happy, but if num or date_cut included something malicious
## your data could be at risk

SELECT * FROM logins WHERE userid = 579;
SELECT * FROM logins WHERE tmstmp > 2014-08-01;


#### Now back to our query template and determine file path for imports

### Practice safe SQL with Psycopg2

In [160]:
query = '''
        COPY logins 
        FROM %(file_path)s
        DELIMITER ','
        CSV;
        '''

In [161]:
folder_path_docker = docker_path + '/logins_data/'
folder_path_local = local_path + '/logins_data/'
for file_name in os.listdir(folder_path_local):
    if file_name.endswith('.csv') and file_name != 'logins01.csv':
        file_path=folder_path_docker+file_name
        cur.execute(query, {'file_path':file_path})
        print('{0} inserted into table.'.format(file_name))

logins08.csv inserted into table.
logins06.csv inserted into table.
logins07.csv inserted into table.
logins05.csv inserted into table.
logins04.csv inserted into table.
logins03.csv inserted into table.
logins02.csv inserted into table.


To use .execute the corrent way, see how in the query string we used 
`%(file_path)s`\
Then in the `.execute()` method we used `{'file_path':file_path}` as a parameter


#### For more on SQL injections check out [bobby-tables.com](http://www.bobby-tables.com/)


### Now, Let's check the total number of records we have right now.

In [162]:
print("Old record count: {}".format(record_count))

cur.execute('SELECT count(*) FROM logins;')
record_count = cur.fetchone()[0]

print("New record count: {}".format(record_count))

Old record count: 10000
New record count: 78588


### Transactions can be rolled back until they're committed

In [163]:
conn.rollback()

cur.execute('SELECT count(*) FROM logins;')
record_count = cur.fetchone()[0]

print("After rollback: {}".format(record_count))

After rollback: 10000


### Don't forget to commit your changes

In [164]:
conn.commit()

### Close your connection

In [165]:
conn.close()

### Using python `with` Statements

In [166]:
query = "SELECT count(*) FROM logins;"
with psycopg2.connect(dbname='temp', host='localhost') as conn:
    with conn.cursor() as curs:
        print("Cursor inside with block: {}".format(curs))
        curs.execute(query)
        record_count = curs.fetchone()[0]
        print(record_count)
    print("Cursor outside with block: {}".format(curs))
    

Cursor inside with block: <cursor object at 0x7f80a3850750; closed: 0>
10000
Cursor outside with block: <cursor object at 0x7f80a3850750; closed: -1>


### Note that the connection is *not* closed automatically:

In [167]:
conn

<connection object at 0x7f80a3879890; dsn: 'dbname=temp host=localhost', closed: 0>

In [168]:
conn.close()
conn

<connection object at 0x7f80a3879890; dsn: 'dbname=temp host=localhost', closed: 1>

### Key Things to Remember about psycopg2

* Connections must be established using an existing database, username, database IP/URL, and maybe passwords
* If you have no created databases, you can connect to Postgres using the dbname 'postgres' to initialize db commands
* Data changes are not actually stored until you choose to commit. This can be done either through `conn.commit()` or setting `autocommit = True`.  Until commited, all transactions is only temporary stored.
* Autocommit = True is necessary to do database commands like CREATE DATABASE.  This is because Postgres does not have temporary transactions at the database level.
* If you ever need to build similar pipelines for other forms of database, there are libraries such PyODBC which operate very similarly.
* SQL connection databases utilizes cursors for data traversal and retrieval.  This is kind of like an iterator in Python.
* Cursor operations typically goes like the following:
    - execute a query
    - fetch rows from query result if it is a SELECT query
    - because it is iterative, previously fetched rows can only be fetched again by rerunning the query
    - close cursor through .close()
* Cursors and Connections must be closed using .close() or else Postgres will lock certain operation on the database/tables to connection is severed. 

# Part 2 RDBMS the why and how:

## Relational Database Management System (RDBMS)

#### It is a persistent data storage system
 - Schema defines the structure of a table or a database
 - Database is composed of a number of user-defined tables
 - Each table has columns (or fields) and rows (or records)
 - A column is of a certain data type such as an integer, text, or date

With a new data source, your first task is typically to understand the schema. 
This will likely take time and conversations with those that gave you access to the database or its data.



![RDBMS Diagram](img/RDBMS_Diagram.png)

#### RDBMS and SQL

 - SQL is the language used to query relational databases
 - **All RDBMS** use SQL and the syntax and keywords are the same for the most part, across systems
 - **SQL is used to interact** with RDBMS, allowing you to create tables, alter tables, insert records, update records, delete records, and query records within and across tables.
 - Even non-relational databases like **Hadoop** usually have a SQL-like interface available.

#### Database Life Cycle (DBLC)

![DBLC](img/DBLC_Diagram.jpg)

#### An inefficient way to store data...

##### A single table with records of customer purchases at an outdoor sports store.

![Inefficient Table](img/Inefficient_Table.jpg)

#### Relational Database Management Systems

 - A RDBMS is a type of database where **data is stored in multiple related tables.**
 - The tables are related through **primary** and **foreign keys**.
 - The same information as shown before in an RDBMS:


#### Primary Keys

 - Every table in a RDBMS has a **primary key**  that uniquely identifies that row
 - Each entry must have a primary key, and primary keys cannot repeat within a table
 - Primary keys are usually integers, often [GUIDs or UUIDs](https://www.guidgenerator.com/online-guid-generator.aspx), but can take other forms


#### Foreign Keys and Table Relationships

 - A **foreign key** is a column that uniquely identifies a column in another table
 - Often, a foreign key in one table is a primary key in another table
 - We can use foreign keys to join tables
 
  ![Foreign Key Diagram](img/Foreign_Key_Diagram.jpg)


#### Entity Relationship Diagram (ERD)

An Entity Relationship Diagram (ERD) represents how each forien key and primary key connects the tables. When using real production data, these diagrams can take up many pages.

![ERD](img/ERD_Diagram.jpg)

#### Database Normalization

##### Store each piece of information in exactly one place
 - Details about a user (address, age) are only stored once (in a users table)
 - Any other table (eg. purchases) where this data might be relevant, only references the user_id
 
![RDBMS](img/RDBMS_Diagram.png)

##### Why is this a good thing?


#### Normal Forms

Normalization is explained through *Normal forms*

 - 1st Normal Form: each attribute has a single value
 - 2nd Normal Form: no dependencies on part of a key
 - 3rd Normal Form: no transitive dependencies


#### Why Not Normalize?

Sometimes databases are not fully normalized.

 - Data structure not known/may change
 - Writing a schema/converting data is hard
 - Simple queries are important
 - Data will not be changed/integrity not important
 - Storage is cheap


![DLBC2](img/DBLC_Diagram2.jpg)
#### Data Science in the DBLC

 - Data Science Operations: querying, aggregating
 - Data Science Implementation: identifying, cleaning, pushing external data sources inside a RDBMS
 - Data Science Design: recommendations on the model, specs on operations
 
 

#### Question:

As a data scientist, what are the advantages of
storing, querying, and maintaining data in a SQL database
over curating your own flat files (e.g. csv files) ?

#### Creating a table with a schema

![Create Table](img/create_table.jpg)

#### Inserting values into a table

![insert into table](img/Insert_into_table.jpg)

#### SQL Queries for table creation / maintenance

Creating a table from query:
```sql
CREATE [TEMPORARY] TABLE table AS <SQL query>;
```
Inserting records in a table:
```sql
INSERT INTO table [(c1,c2,c3,…)] VALUES (v1,v2,v3,…);
```
Updating records:
```sql
UPDATE table SET c1=v1,c2=v2,… WHERE cX=vX;
```
Delete records:
```sql
DELETE FROM table WHERE cX=vX;
```
Change model (add, drop, modify columns):
```sql
ALTER TABLE table [DROP/ADD/ALTER] column [datatype];
```
Delete a table:
```sql
DROP TABLE table;
```

![DLBC](img/DBLC_Diagram.jpg)

#### Question?

How many rows would result in
(inner join, left join, right join, full outer join) on `department_id` ?


#### Order of Evaluation of a SQL SELECT Statement

1. FROM + JOIN: first the product of all tables is formed
2. WHERE: the where clause filters rows that do not meet the search condition
3. GROUP BY + (COUNT, SUM, etc): the rows are grouped using the columns in the group by clause and the aggregation functions are applied on the grouping
4. HAVING: like the WHERE clause, but can be applied after aggregation
5. SELECT: the targeted list of columns are evaluated and returned
6. DISTINCT: duplicate rows are eliminated
7. ORDER BY: the resulting rows are sorted


#### Order of Evaluation of a SQL SELECT Statement

![Order of Evaluation](img/Order_of_Evaluation.jpg)

#### Transactions

Suppose you update multiple records in a single statement.
One update fails.
\
What should happen?
\
What does happen?


Database statements can be groups into a transaction.
End transaction with `commit` or `rollback`.\
Default interactive behavior is autocommit.
\
\
**Transactions are:**
 - Atomic
 - Consistent
 - Isolated
 - Durable


#### Excecution Plan

When troubleshooting a query, you may want to figure out what your query is in fact doing. Most SQL systems have their own internal query optomiser, so if the query you write is not in the most efficient form, the sql system will optomize it. Sometimes still your query will fail. You can use the Execution plan to troubleshoot this.

![Exec](img/exec_plan.png)

If you are concerned about running too large a query, you can run your execution plan prior to running the query. Running to large of a query may be hard to accomplish using the datasets you find online, but can be a real concern when working with real world data.

## Lecture Objectives

#### Part 1 SQL with python (psycopg2):

- Connect to a database from within a python program and run queries
- Understand psycopg2's cursors and commits
- Generate dynamic queries

#### Part 2 RDBMS the why and how:

 - Why an RDBMS?
 - The database lifecycle (and where the data scientist fits!)
 - Primary keys, foriegn keys, and the ERD
 - Creating and maintaining tables
 - Order of Evaluation of a SQL SELECT Statement
 - Transactions
 - SQL Execution Plan